In [8]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.optim import Adam
import time


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
    ])

trainset = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=True, download=True,transform=transform)
testset = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=False, transform=transform)

train_loader = DataLoader(trainset, batch_size=256, shuffle=True)
test_loader = DataLoader(testset, batch_size=256, shuffle=False)

Files already downloaded and verified


In [10]:
for images, labels in train_loader:
  print(images.size(), labels.size())
  break

torch.Size([256, 3, 32, 32]) torch.Size([256])


In [11]:
class CustomCNNCifar(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomCNNCifar, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=2, padding=1, padding_mode='zeros')
        #16x16x 8
        self.relu1 = nn.LeakyReLU()
        
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1, padding_mode='zeros')
        #8x8x16
        self.relu2 = nn.LeakyReLU()
        
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1, padding_mode='zeros')
        #4x4x32
        self.relu3 = nn.LeakyReLU()
        
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1, padding_mode='zeros')
        #2x2x64
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) 
        #1x1x64
        self.relu4 = nn.LeakyReLU()

        self.flatten = nn.Flatten()        
                
        self.fc1 = nn.Linear(64, num_classes)
       
        self.log_softmax = nn.LogSoftmax(dim=1)  # Softmax activation for classification



    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        
        x = self.conv3(x)
        x = self.relu3(x)
        
        x = self.conv4(x)
        x = self.pool1(x)
        x = self.relu4(x)
        
        x = self.flatten(x)
        
        x = self.fc1(x)
        
        x = self.log_softmax(x)  # Apply softmax for classification
        return x


# Create an instance of the SimpleCNNReduced model
model = CustomCNNCifar(num_classes=10)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Calculate the total number of trainable parameters
total_params_reduced = count_parameters(model)
print(f"Total trainable parameters in the reduced model: {total_params_reduced}")

Total trainable parameters in the reduced model: 25178


In [12]:
print(model.to(device))

CustomCNNCifar(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu1): LeakyReLU(negative_slope=0.01)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu2): LeakyReLU(negative_slope=0.01)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu3): LeakyReLU(negative_slope=0.01)
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu4): LeakyReLU(negative_slope=0.01)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=64, out_features=10, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)


In [13]:
model = model.to(device)
optimizer = Adam(model.parameters())
criterion = nn.NLLLoss()

num_epochs = 10
batch_loss = 0
cum_epoch_loss = 0

start_time = time.time()

for e in range(num_epochs):
  cum_epoch_loss = 0

  for batch, (images, labels) in enumerate(train_loader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()

    batch_loss += loss.item()
    print(f'Epoch({e+1}/{num_epochs} : Batch number({batch}/{len(train_loader)}) : Batch loss : {loss.item()}')

  print(f'Training loss : {batch_loss/len(train_loader)}')

end_time = time.time()
elapsed_time = end_time - start_time
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

print(f"Elapsed Time: {minutes} minutes and {seconds} seconds")


Epoch(1/10 : Batch number(1/196) : Batch loss : 2.3147947788238525
Epoch(1/10 : Batch number(2/196) : Batch loss : 2.3038861751556396
Epoch(1/10 : Batch number(3/196) : Batch loss : 2.3040385246276855
Epoch(1/10 : Batch number(4/196) : Batch loss : 2.297102451324463
Epoch(1/10 : Batch number(5/196) : Batch loss : 2.295910596847534
Epoch(1/10 : Batch number(6/196) : Batch loss : 2.30352783203125
Epoch(1/10 : Batch number(7/196) : Batch loss : 2.293605327606201
Epoch(1/10 : Batch number(8/196) : Batch loss : 2.2985410690307617
Epoch(1/10 : Batch number(9/196) : Batch loss : 2.293850898742676
Epoch(1/10 : Batch number(10/196) : Batch loss : 2.2925217151641846
Epoch(1/10 : Batch number(11/196) : Batch loss : 2.284846782684326
Epoch(1/10 : Batch number(12/196) : Batch loss : 2.287590980529785
Epoch(1/10 : Batch number(13/196) : Batch loss : 2.274374485015869
Epoch(1/10 : Batch number(14/196) : Batch loss : 2.270517587661743
Epoch(1/10 : Batch number(15/196) : Batch loss : 2.275195837020874


Epoch(1/10 : Batch number(123/196) : Batch loss : 1.79990816116333
Epoch(1/10 : Batch number(124/196) : Batch loss : 1.6690545082092285
Epoch(1/10 : Batch number(125/196) : Batch loss : 1.7050731182098389
Epoch(1/10 : Batch number(126/196) : Batch loss : 1.7004599571228027
Epoch(1/10 : Batch number(127/196) : Batch loss : 1.7217626571655273
Epoch(1/10 : Batch number(128/196) : Batch loss : 1.7526748180389404
Epoch(1/10 : Batch number(129/196) : Batch loss : 1.701839804649353
Epoch(1/10 : Batch number(130/196) : Batch loss : 1.681122064590454
Epoch(1/10 : Batch number(131/196) : Batch loss : 1.6780204772949219
Epoch(1/10 : Batch number(132/196) : Batch loss : 1.7644504308700562
Epoch(1/10 : Batch number(133/196) : Batch loss : 1.72640860080719
Epoch(1/10 : Batch number(134/196) : Batch loss : 1.7782948017120361
Epoch(1/10 : Batch number(135/196) : Batch loss : 1.7023776769638062
Epoch(1/10 : Batch number(136/196) : Batch loss : 1.6756372451782227
Epoch(1/10 : Batch number(137/196) : Bat

Epoch(2/10 : Batch number(49/196) : Batch loss : 1.5169109106063843
Epoch(2/10 : Batch number(50/196) : Batch loss : 1.558699607849121
Epoch(2/10 : Batch number(51/196) : Batch loss : 1.581810474395752
Epoch(2/10 : Batch number(52/196) : Batch loss : 1.5899333953857422
Epoch(2/10 : Batch number(53/196) : Batch loss : 1.6459252834320068
Epoch(2/10 : Batch number(54/196) : Batch loss : 1.659014105796814
Epoch(2/10 : Batch number(55/196) : Batch loss : 1.5769731998443604
Epoch(2/10 : Batch number(56/196) : Batch loss : 1.6264592409133911
Epoch(2/10 : Batch number(57/196) : Batch loss : 1.5930358171463013
Epoch(2/10 : Batch number(58/196) : Batch loss : 1.440381646156311
Epoch(2/10 : Batch number(59/196) : Batch loss : 1.5635974407196045
Epoch(2/10 : Batch number(60/196) : Batch loss : 1.597303032875061
Epoch(2/10 : Batch number(61/196) : Batch loss : 1.5536259412765503
Epoch(2/10 : Batch number(62/196) : Batch loss : 1.5815671682357788
Epoch(2/10 : Batch number(63/196) : Batch loss : 1.64

Epoch(2/10 : Batch number(173/196) : Batch loss : 1.4527288675308228
Epoch(2/10 : Batch number(174/196) : Batch loss : 1.4442799091339111
Epoch(2/10 : Batch number(175/196) : Batch loss : 1.422390103340149
Epoch(2/10 : Batch number(176/196) : Batch loss : 1.4615626335144043
Epoch(2/10 : Batch number(177/196) : Batch loss : 1.5851683616638184
Epoch(2/10 : Batch number(178/196) : Batch loss : 1.4909203052520752
Epoch(2/10 : Batch number(179/196) : Batch loss : 1.438110113143921
Epoch(2/10 : Batch number(180/196) : Batch loss : 1.4288228750228882
Epoch(2/10 : Batch number(181/196) : Batch loss : 1.4336010217666626
Epoch(2/10 : Batch number(182/196) : Batch loss : 1.53273344039917
Epoch(2/10 : Batch number(183/196) : Batch loss : 1.444848895072937
Epoch(2/10 : Batch number(184/196) : Batch loss : 1.5101364850997925
Epoch(2/10 : Batch number(185/196) : Batch loss : 1.500036597251892
Epoch(2/10 : Batch number(186/196) : Batch loss : 1.4619642496109009
Epoch(2/10 : Batch number(187/196) : Bat

Epoch(3/10 : Batch number(100/196) : Batch loss : 1.4792228937149048
Epoch(3/10 : Batch number(101/196) : Batch loss : 1.4139404296875
Epoch(3/10 : Batch number(102/196) : Batch loss : 1.3321471214294434
Epoch(3/10 : Batch number(103/196) : Batch loss : 1.2529613971710205
Epoch(3/10 : Batch number(104/196) : Batch loss : 1.41499662399292
Epoch(3/10 : Batch number(105/196) : Batch loss : 1.354818344116211
Epoch(3/10 : Batch number(106/196) : Batch loss : 1.3869868516921997
Epoch(3/10 : Batch number(107/196) : Batch loss : 1.436919093132019
Epoch(3/10 : Batch number(108/196) : Batch loss : 1.4649156332015991
Epoch(3/10 : Batch number(109/196) : Batch loss : 1.4289473295211792
Epoch(3/10 : Batch number(110/196) : Batch loss : 1.3558012247085571
Epoch(3/10 : Batch number(111/196) : Batch loss : 1.3503615856170654
Epoch(3/10 : Batch number(112/196) : Batch loss : 1.3773635625839233
Epoch(3/10 : Batch number(113/196) : Batch loss : 1.4496979713439941
Epoch(3/10 : Batch number(114/196) : Batc

...

Epoch(7/10 : Batch number(178/196) : Batch loss : 1.2376997470855713
Epoch(7/10 : Batch number(179/196) : Batch loss : 1.186915636062622
Epoch(7/10 : Batch number(180/196) : Batch loss : 1.2293891906738281
Epoch(7/10 : Batch number(181/196) : Batch loss : 1.1543729305267334
Epoch(7/10 : Batch number(182/196) : Batch loss : 1.1503785848617554
Epoch(7/10 : Batch number(183/196) : Batch loss : 1.2015900611877441
Epoch(7/10 : Batch number(184/196) : Batch loss : 1.2302755117416382
Epoch(7/10 : Batch number(185/196) : Batch loss : 1.2149162292480469
Epoch(7/10 : Batch number(186/196) : Batch loss : 1.2226104736328125
Epoch(7/10 : Batch number(187/196) : Batch loss : 1.2494393587112427
Epoch(7/10 : Batch number(188/196) : Batch loss : 1.336382269859314
Epoch(7/10 : Batch number(189/196) : Batch loss : 1.2098804712295532
Epoch(7/10 : Batch number(190/196) : Batch loss : 1.3300981521606445
Epoch(7/10 : Batch number(191/196) : Batch loss : 1.2830591201782227
Epoch(7/10 : Batch number(192/196) :

Epoch(8/10 : Batch number(107/196) : Batch loss : 1.1974022388458252
Epoch(8/10 : Batch number(108/196) : Batch loss : 1.1515333652496338
Epoch(8/10 : Batch number(109/196) : Batch loss : 1.1943079233169556
Epoch(8/10 : Batch number(110/196) : Batch loss : 1.235968828201294
Epoch(8/10 : Batch number(111/196) : Batch loss : 1.1890501976013184
Epoch(8/10 : Batch number(112/196) : Batch loss : 1.2084381580352783
Epoch(8/10 : Batch number(113/196) : Batch loss : 1.244848608970642
Epoch(8/10 : Batch number(114/196) : Batch loss : 1.1932138204574585
Epoch(8/10 : Batch number(115/196) : Batch loss : 1.238560676574707
Epoch(8/10 : Batch number(116/196) : Batch loss : 1.2150744199752808
Epoch(8/10 : Batch number(117/196) : Batch loss : 1.2710299491882324
Epoch(8/10 : Batch number(118/196) : Batch loss : 1.1215792894363403
Epoch(8/10 : Batch number(119/196) : Batch loss : 1.239805817604065
Epoch(8/10 : Batch number(120/196) : Batch loss : 1.2730873823165894
Epoch(8/10 : Batch number(121/196) : B

Epoch(9/10 : Batch number(32/196) : Batch loss : 1.1607081890106201
Epoch(9/10 : Batch number(33/196) : Batch loss : 1.179887056350708
Epoch(9/10 : Batch number(34/196) : Batch loss : 1.142483115196228
Epoch(9/10 : Batch number(35/196) : Batch loss : 1.083911657333374
Epoch(9/10 : Batch number(36/196) : Batch loss : 1.3069394826889038
Epoch(9/10 : Batch number(37/196) : Batch loss : 1.1296130418777466
Epoch(9/10 : Batch number(38/196) : Batch loss : 1.216125726699829
Epoch(9/10 : Batch number(39/196) : Batch loss : 1.104918122291565
Epoch(9/10 : Batch number(40/196) : Batch loss : 1.1965383291244507
Epoch(9/10 : Batch number(41/196) : Batch loss : 1.317369818687439
Epoch(9/10 : Batch number(42/196) : Batch loss : 1.2037253379821777
Epoch(9/10 : Batch number(43/196) : Batch loss : 1.1294755935668945
Epoch(9/10 : Batch number(44/196) : Batch loss : 1.135337471961975
Epoch(9/10 : Batch number(45/196) : Batch loss : 1.2152398824691772
Epoch(9/10 : Batch number(46/196) : Batch loss : 1.0902

Epoch(9/10 : Batch number(156/196) : Batch loss : 1.1481082439422607
Epoch(9/10 : Batch number(157/196) : Batch loss : 1.2249988317489624
Epoch(9/10 : Batch number(158/196) : Batch loss : 1.1699765920639038
Epoch(9/10 : Batch number(159/196) : Batch loss : 1.2172434329986572
Epoch(9/10 : Batch number(160/196) : Batch loss : 1.1386114358901978
Epoch(9/10 : Batch number(161/196) : Batch loss : 1.2626432180404663
Epoch(9/10 : Batch number(162/196) : Batch loss : 1.1767973899841309
Epoch(9/10 : Batch number(163/196) : Batch loss : 1.2134909629821777
Epoch(9/10 : Batch number(164/196) : Batch loss : 1.2614684104919434
Epoch(9/10 : Batch number(165/196) : Batch loss : 1.2419887781143188
Epoch(9/10 : Batch number(166/196) : Batch loss : 1.1412702798843384
Epoch(9/10 : Batch number(167/196) : Batch loss : 1.258737325668335
Epoch(9/10 : Batch number(168/196) : Batch loss : 1.1889246702194214
Epoch(9/10 : Batch number(169/196) : Batch loss : 1.2472323179244995
Epoch(9/10 : Batch number(170/196) 

Epoch(10/10 : Batch number(81/196) : Batch loss : 1.2191524505615234
Epoch(10/10 : Batch number(82/196) : Batch loss : 1.1234112977981567
Epoch(10/10 : Batch number(83/196) : Batch loss : 1.0966941118240356
Epoch(10/10 : Batch number(84/196) : Batch loss : 0.9963945746421814
Epoch(10/10 : Batch number(85/196) : Batch loss : 1.1580830812454224
Epoch(10/10 : Batch number(86/196) : Batch loss : 1.1036823987960815
Epoch(10/10 : Batch number(87/196) : Batch loss : 1.0779255628585815
Epoch(10/10 : Batch number(88/196) : Batch loss : 1.2072559595108032
Epoch(10/10 : Batch number(89/196) : Batch loss : 1.1685242652893066
Epoch(10/10 : Batch number(90/196) : Batch loss : 1.2103415727615356
Epoch(10/10 : Batch number(91/196) : Batch loss : 1.2209794521331787
Epoch(10/10 : Batch number(92/196) : Batch loss : 1.234804391860962
Epoch(10/10 : Batch number(93/196) : Batch loss : 1.1581239700317383
Epoch(10/10 : Batch number(94/196) : Batch loss : 1.1016087532043457
Epoch(10/10 : Batch number(95/196) 

In [14]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(test_loader,1):

        logps = model(images)
        output = torch.exp(logps)

        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(test_loader)})')

        # if batch == 5:
         # break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/40)
Batch (2/40)
Batch (3/40)
Batch (4/40)
Batch (5/40)
Batch (6/40)
Batch (7/40)
Batch (8/40)
Batch (9/40)
Batch (10/40)
Batch (11/40)
Batch (12/40)
Batch (13/40)
Batch (14/40)
Batch (15/40)
Batch (16/40)
Batch (17/40)
Batch (18/40)
Batch (19/40)
Batch (20/40)
Batch (21/40)
Batch (22/40)
Batch (23/40)
Batch (24/40)
Batch (25/40)
Batch (26/40)
Batch (27/40)
Batch (28/40)
Batch (29/40)
Batch (30/40)
Batch (31/40)
Batch (32/40)
Batch (33/40)
Batch (34/40)
Batch (35/40)
Batch (36/40)
Batch (37/40)
Batch (38/40)
Batch (39/40)
Batch (40/40)
Accuracy of the model on 10000 test images: 55.98% 
